In [22]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import QAGenerationChain
from langchain_huggingface import HuggingFacePipeline
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from torch import bfloat16
import pandas as pd

ModuleNotFoundError: No module named 'hdbscan._hdbscan_linkage'

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=50, chunk_size=300)
loader = TextLoader("./files/ejemplo.txt", encoding="utf-8")
doc = loader.load()[0]

In [5]:
import torch
torch.cuda.empty_cache()

In [16]:
llm = HuggingFacePipeline.from_model_id(
    model_id="meta-llama/Meta-Llama-3-8B",
    device_map="auto",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.1,
        "do_sample": True
    },
)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [17]:
chain = QAGenerationChain.from_llm(llm=llm, text_splitter=text_splitter)


In [18]:
qa = chain.invoke(doc.page_content)
print(qa[1])

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
chain

QAGenerationChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['text'], template='You are a smart assistant designed to help high school teachers come up with reading comprehension questions.\nGiven a piece of text, you must come up with a question and answer pair that can be used to test a student\'s reading comprehension abilities.\nWhen coming up with this question/answer pair, you must respond in the following format:\n```\n{{\n    "question": "$YOUR_QUESTION_HERE",\n    "answer": "$THE_ANSWER_HERE"\n}}\n```\n\nEverything between the ``` must be valid json.\n\nPlease come up with a question/answer pair, in the specified JSON format, for the following text:\n----------------\n{text}'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x000001EB4572F5F0>, model_id='meta-llama/Llama-2-7b-chat-hf', model_kwargs={'device_map': 'auto'}, pipeline_kwargs={'max_new_tokens': 100, 'top_k': 50, 'temperature': 0.1, 'do_sample

QAGenerationChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['text'], template='You are a smart assistant designed to help high school teachers come up with reading comprehension questions.\nGiven a piece of text, you must come up with a question and answer pair that can be used to test a student\'s reading comprehension abilities.\nWhen coming up with this question/answer pair, you must respond in the following format:\n```\n{{\n    "question": "$YOUR_QUESTION_HERE",\n    "answer": "$THE_ANSWER_HERE"\n}}\n```\n\nEverything between the ``` must be valid json.\n\nPlease come up with a question/answer pair, in the specified JSON format, for the following text:\n----------------\n{text}'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x000001EB4572F5F0>, model_id='meta-llama/Llama-2-7b-chat-hf', model_kwargs={'device_map': 'auto'}, pipeline_kwargs={'max_new_tokens': 100, 'top_k': 50, 'temperature': 0.1, 'do_sample': True})), text_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x000001E5CB39C620>)

In [ ]:
model_id = 'meta-llama/Meta-Llama-3-8B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
nf4_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=bfloat16)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, quantization_config=nf4_config, device_map='auto')
model.eval()

generator = pipeline(model=model, tokenizer=tokenizer, task='text-generation', temperature=0.1, max_new_tokens=500, repetition_penalty=1.1)

In [23]:
def new_prompt(text):
    prompt = f"""
    <s>[INST] <<SYS>>
    Eres un asistente inteligente diseñado para ayudar a los profesores de secundaria a crear preguntas de comprensión lectora.
    <</SYS>>
    Dado este fragmento de texto, debes formular un par de pregunta y respuesta que pueda usarse para evaluar las habilidades de comprensión lectora de un estudiante.
    Al formular este par de pregunta/respuesta, debes responder en el siguiente formato:
    ```
    {{
        "question": "$TU_PREGUNTA_AQUÍ",
        "answer": "$LA_RESPUESTA_AQUÍ"
    }}
    ```
    
    Todo lo que esté entre ``` debe ser un JSON válido.
    Por favor, formula un par de pregunta/respuesta, en el formato JSON especificado, para el siguiente texto:
    ----------------
    {text}
    [/INST]
    """
    return prompt


In [25]:
with open('./files/ejemplo.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [26]:
prompt = new_prompt(text)

In [30]:
import transformers
import torch

model_id = "meta-llama/Llama-2-7b-chat-hf"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16},  max_new_tokens = 100
)
pipeline(prompt)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]